In [1]:
from src.nws import NWSClient

client = NWSClient()

In [6]:
# For each station, I want to get the cli report and determine what time the max temp was achieved at
# Since the summaries are for different times, this involves going through the summaries, grouping by summary_date, 
# and then finding the latest max temp for each day.

from src.params import *

station = StationID.MDW

# Get all the cli reports
cli_reports = client.get_all_cli_data(station)
cli_reports

[CLI(id='7ad24812-f9e3-4540-8f82-94b3fedac48d', url=URL('https://api.weather.gov/products/7ad24812-f9e3-4540-8f82-94b3fedac48d'), issuance_time=datetime.datetime(2024, 11, 12, 6, 44, tzinfo=datetime.timezone.utc), product_code='CLI', product_name='Climatological Report (Daily)', issuing_office='KLOT', report=ParsedCLIReport(summary_date=datetime.date(2024, 11, 11), raw_text='\n000\nCDUS43 KLOT 120644\nCLIMDW\n\nCLIMATE REPORT \nNATIONAL WEATHER SERVICE CHICAGO IL\n1244 AM CST TUE NOV 12 2024\n\n...................................\n\n...THE CHICAGO-MIDWAY CLIMATE SUMMARY FOR NOVEMBER 11 2024...\n\nCLIMATE NORMAL PERIOD: 1991 TO 2020\nCLIMATE RECORD PERIOD: 1928 TO 2024\n\n\nWEATHER ITEM   OBSERVED TIME   RECORD YEAR NORMAL DEPARTURE LAST      \n                VALUE   (LST)  VALUE       VALUE  FROM      YEAR     \n                                                  NORMAL           \n...................................................................\nTEMPERATURE (F)                      

In [7]:
from pprint import pprint

len(cli_reports)
pprint(cli_reports[12].report.raw_text)

('\n'
 '000\n'
 'CDUS43 KLOT 060637\n'
 'CLIMDW\n'
 '\n'
 'CLIMATE REPORT \n'
 'NATIONAL WEATHER SERVICE CHICAGO IL\n'
 '1237 AM CST WED NOV 06 2024\n'
 '\n'
 '...................................\n'
 '\n'
 '...THE CHICAGO-MIDWAY CLIMATE SUMMARY FOR NOVEMBER 5 2024...\n'
 '\n'
 'CLIMATE NORMAL PERIOD: 1991 TO 2020\n'
 'CLIMATE RECORD PERIOD: 1928 TO 2024\n'
 '\n'
 '\n'
 'WEATHER ITEM   OBSERVED TIME   RECORD YEAR NORMAL DEPARTURE LAST      \n'
 '                VALUE   (LST)  VALUE       VALUE  FROM      YEAR     \n'
 '                                                  NORMAL           \n'
 '...................................................................\n'
 'TEMPERATURE (F)                                                          \n'
 ' YESTERDAY                                                           \n'
 '  MAXIMUM         68   2:36 PM  79    1978  54     14       61        \n'
 '  MINIMUM         61  11:59 PM  11    1951  39     22       36        \n'
 '  AVERAGE         65    

In [8]:
from collections import defaultdict

grouped_by_summary_dates = defaultdict(list)
for cli in cli_reports:
    grouped_by_summary_dates[cli.report.summary_date].append(cli)

# Now take the latest summary for each day
latest_summaries = {summary_date: max(clis, key=lambda x: x.issuance_time) for summary_date, clis in grouped_by_summary_dates.items()}
latest_summaries

{datetime.date(2024, 11, 11): CLI(id='7ad24812-f9e3-4540-8f82-94b3fedac48d', url=URL('https://api.weather.gov/products/7ad24812-f9e3-4540-8f82-94b3fedac48d'), issuance_time=datetime.datetime(2024, 11, 12, 6, 44, tzinfo=datetime.timezone.utc), product_code='CLI', product_name='Climatological Report (Daily)', issuing_office='KLOT', report=ParsedCLIReport(summary_date=datetime.date(2024, 11, 11), raw_text='\n000\nCDUS43 KLOT 120644\nCLIMDW\n\nCLIMATE REPORT \nNATIONAL WEATHER SERVICE CHICAGO IL\n1244 AM CST TUE NOV 12 2024\n\n...................................\n\n...THE CHICAGO-MIDWAY CLIMATE SUMMARY FOR NOVEMBER 11 2024...\n\nCLIMATE NORMAL PERIOD: 1991 TO 2020\nCLIMATE RECORD PERIOD: 1928 TO 2024\n\n\nWEATHER ITEM   OBSERVED TIME   RECORD YEAR NORMAL DEPARTURE LAST      \n                VALUE   (LST)  VALUE       VALUE  FROM      YEAR     \n                                                  NORMAL           \n...................................................................\nTEMPERAT

In [9]:
for summary_date, cli in sorted(latest_summaries.items(), key=lambda x: x[0], reverse=True):
    print(f"Summary Date: {summary_date}")
    print(f"Max Temp: {cli.report.max_temp}")
    print(f"Time of Max Temp: {cli.report.max_temp_time}")
    print(f"URL: {cli.url}")
    print(f"")

Summary Date: 2024-11-11
Max Temp: 59
Time of Max Temp: 2024-11-11 13:55:00
URL: https://api.weather.gov/products/7ad24812-f9e3-4540-8f82-94b3fedac48d

Summary Date: 2024-11-10
Max Temp: 64
Time of Max Temp: 2024-11-10 13:40:00
URL: https://api.weather.gov/products/e75e8451-46da-4540-8433-ea7fd53686a8

Summary Date: 2024-11-09
Max Temp: 56
Time of Max Temp: 2024-11-09 14:13:00
URL: https://api.weather.gov/products/c8ac971b-b2b8-4d63-bb01-7f361b1008c1

Summary Date: 2024-11-08
Max Temp: 64
Time of Max Temp: 2024-11-08 14:48:00
URL: https://api.weather.gov/products/c0c888b9-def9-43b8-873a-04e7b17bff5a

Summary Date: 2024-11-07
Max Temp: 60
Time of Max Temp: 2024-11-07 15:31:00
URL: https://api.weather.gov/products/395872fe-5fca-4e76-9de0-471b527d33c5

Summary Date: 2024-11-06
Max Temp: 61
Time of Max Temp: 2024-11-06 00:50:00
URL: https://api.weather.gov/products/188fdab0-4eee-43f1-a91f-168dab5ae0e8

Summary Date: 2024-11-05
Max Temp: 68
Time of Max Temp: 2024-11-05 14:36:00
URL: https:/